##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 数据集集合

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/datasets/dataset_collections"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/datasets/dataset_collections.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/datasets/dataset_collections.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/datasets/dataset_collections.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a> </td>
</table>

## 概述

数据集集合提供了一种简单的方法，可以将任意数量的现有 TFDS 数据集组合在一起，并对它们执行简单的运算。

它们可能会很有用，例如，将与同一任务相关的不同数据集组合在一起，或者便于对固定数量的不同任务中的模型进行[基准测试](https://ruder.io/nlp-benchmarking/)。

## 安装

首先，安装几个软件包：

In [ ]:
# Use tfds-nightly to ensure access to the latest features.
!pip install -q tfds-nightly tensorflow
!pip install -U conllu

将 TensorFlow 和 TensorFlow Datasets 软件包导入您的开发环境：

In [ ]:
import pprint

import tensorflow as tf
import tensorflow_datasets as tfds

数据集集合提供了一种简单的方式，可以将 Tensorflow Datasets 中任意数量的现有数据集组合在一起，并对它们执行简单的运算。

它们可能会很有用，例如，将与同一任务相关的不同数据集组合在一起，或者便于对固定数量的不同任务中的模型进行[基准测试](https://ruder.io/nlp-benchmarking/)。

## 查找可用的数据集集合

所有数据集集合构建工具都是 `tfds.core.dataset_collection_builder.DatasetCollection` 的子类。

要获取可用构建工具的列表，请使用 `tfds.list_dataset_collections()`。


In [ ]:
tfds.list_dataset_collections()

## 加载并检查数据集集合

加载数据集集合的最简单方式是使用 [`tfds.dataset_collection`](https://tensorflow.google.cn/datasets/api_docs/python/tfds/dataset_collection) 命令实例化 `DatasetCollectionLoader` 对象。


In [ ]:
collection_loader = tfds.dataset_collection('xtreme')

可以按照与加载TFDS 数据集相同的语法加载特定的数据集集合版本：

In [ ]:
collection_loader = tfds.dataset_collection('xtreme:1.0.0')

数据集集合加载程序可以显示有关集合的信息：

In [ ]:
collection_loader.print_info()

数据集加载程序还可以显示有关集合中包含的数据集的信息：

In [ ]:
collection_loader.print_datasets()

### 从数据集集合加载数据集

从集合中加载一个数据集的最简单方式是使用 `DatasetCollectionLoader` 对象的 `load_dataset` 方法，此方法通过调用 [`tfds.load`](https://tensorflow.google.cn/datasets/api_docs/python/tfds/load) 加载所需的数据集。

此调用返回拆分名称的字典和相应的 `tf.data.Dataset`：

In [ ]:
splits = collection_loader.load_dataset("ner")

pprint.pprint(splits)

`load_dataset` 接受以下可选形参：

- `split`：要加载的拆分。它接受单个拆分 (`split="test"`) 或拆分列表：(`split=["train", "test"]`)。如果未指定，它将加载给定数据集的所有拆分。
- `loader_kwargs`：要传递给 `tfds.load` 函数的关键字实参。有关不同加载选项的全面概览，请参阅 [`tfds.load`](https://tensorflow.google.cn/datasets/api_docs/python/tfds/load) 文档。

### 从数据集集合加载多个数据集

从集合中加载多个数据集的最简单方式是使用 `DatasetCollectionLoader` 对象的 `load_dataset` 方法，此方法通过调用 [`tfds.load`](https://tensorflow.google.cn/datasets/api_docs/python/tfds/load) 加载所需的数据集。

它返回一个数据集名称字典，每个字典都与一个拆分名称字典和相应的 `tf.data.Dataset` 相关联，如下例所示：

In [ ]:
datasets = collection_loader.load_datasets(['xnli', 'bucc'])

pprint.pprint(datasets)

`load_all_datasets` 方法加载给定集合的*所有*可用数据集：

In [ ]:
all_datasets = collection_loader.load_all_datasets()

pprint.pprint(all_datasets)

`load_datasets` 方法接受以下可选形参：

- `split`：要加载的拆分。它接受单个拆分 (`split="test"`) 或拆分列表：(`split=["train", "test"]`)。如果未指定，它将加载给定数据集的所有拆分。
- `loader_kwargs`：要传递给 `tfds.load` 函数的关键字实参。有关不同加载选项的全面概览，请参阅 [`tfds.load`](https://tensorflow.google.cn/datasets/api_docs/python/tfds/load) 文档。

### 制定 `loader_kwargs`

`loader_kwargs` 是要传递给 [`tfds.load`](https://tensorflow.google.cn/datasets/api_docs/python/tfds/load) 函数的可选关键字实参。可以通过三种方式指定这些实参：

1. 初始化 `DatasetCollectionLoader` 类时：

In [ ]:
collection_loader = tfds.dataset_collection('xtreme', loader_kwargs=dict(split='train', batch_size=10, try_gcs=False))

1. 使用 `DatasetCollectioLoader` 的 `set_loader_kwargs` 方法：

In [ ]:
collection_loader.set_loader_kwargs(dict(split='train', batch_size=10, try_gcs=False))

1. 作为 `load_dataset`、`load_datasets` 和 `load_all_datasets` 方法的可选形参。

In [ ]:
dataset = collection_loader.load_dataset('ner', loader_kwargs=dict(split='train', batch_size=10, try_gcs=False))

### 反馈

我们一直在致力于改进数据集创建工作流，但只有在充分了解存在哪些问题后才能有效地实施改进。您在创建数据集时遇到了哪些问题和错误？是否存在令您困惑的部分、样板问题，以及首次创建失败的经历？请在 [GitHub](https://github.com/tensorflow/datasets/issues) 上分享您的反馈。